***GENERATED CODE FOR lifeinsurancetrain PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Info_2', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Info_2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'A1', 'max': 'E1', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Info_2'}, {'feature_label': 'Product_Info_2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Info_2')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run lifeinsurancetrainHooks.ipynb
try:
	#sourcePreExecutionHook()

	lifeinsurancetrainhdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1676970990.csv', 'filename': 'tempFile1676970990.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(lifeinsurancetrainhdfs)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run lifeinsurancetrainHooks.ipynb
try:
	#transformationPreExecutionHook()

	lifeinsurancetrainautofe = TransformationMain.run(lifeinsurancetrainhdfs,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "341.02", "stddev": "195.44", "min": "2", "max": "683", "missing": "0"}, "updatedLabel": "Id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.02", "stddev": "0.15", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Product_Info_1"}, {"transformationsData": [{"feature_label": "Product_Info_2", "transformation_label": "String Indexer"}], "feature": "Product_Info_2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A1", "max": "E1", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Info_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "24.4", "stddev": "5.32", "min": "4", "max": "37", "missing": "0"}, "updatedLabel": "Product_Info_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.33", "stddev": "0.28", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product_Info_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.0", "stddev": "0.06", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.65", "stddev": "0.76", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product_Info_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.02", "stddev": "0.18", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Product_Info_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ins_Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.41", "stddev": "0.19", "min": "0.0", "max": "0.880597015", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ins_Age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ht", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.71", "stddev": "0.08", "min": "0.472727273", "max": "0.872727273", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ht"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Wt", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.29", "stddev": "0.09", "min": "0.121338912", "max": "0.623430962", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Wt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BMI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.47", "stddev": "0.12", "min": "0.224087006", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BMI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.08", "stddev": "0.08", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "8.37", "stddev": "4.54", "min": "1", "max": "36", "missing": "0"}, "updatedLabel": "Employment_Info_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.36", "stddev": "0.77", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "Employment_Info_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "438", "mean": "0.01", "stddev": "0.02", "min": "0.0", "max": "0.3", "missing": "62"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.16", "stddev": "0.37", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "Employment_Info_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Employment_Info_6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "403", "mean": "0.37", "stddev": "0.36", "min": "0.0", "max": "1.0", "missing": "97"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Employment_Info_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.21", "stddev": "0.42", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.01", "stddev": "0.09", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.81", "stddev": "2.65", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "InsuredInfo_3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_4", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.86", "stddev": "0.35", "min": "2", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_5", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.04", "stddev": "0.27", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_6", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.41", "stddev": "0.49", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "InsuredInfo_6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuredInfo_7", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.03", "stddev": "0.25", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "InsuredInfo_7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Insurance_History_1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.74", "stddev": "0.44", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Insurance_History_1"}]}))

	#transformationPostExecutionHook(lifeinsurancetrainautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run lifeinsurancetrainHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(lifeinsurancetrainautofe, ["Id", "Product_Info_1", "Product_Info_3", "Product_Info_4", "Product_Info_5", "Product_Info_6", "Product_Info_7", "Ins_Age", "Ht", "Wt", "BMI", "Employment_Info_1", "Employment_Info_2", "Employment_Info_3", "Employment_Info_4", "Employment_Info_5", "Employment_Info_6", "InsuredInfo_1", "InsuredInfo_2", "InsuredInfo_3", "InsuredInfo_4", "InsuredInfo_5", "InsuredInfo_6", "InsuredInfo_7", "Product_Info_2_stringindexer"], "Insurance_History_1")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

